# Introduction

### Competition: [Titanic Kaggle](https://www.kaggle.com/c/titanic/overview)

This is notebook contains a simple data science project framework, for learning and portfolio construction purposes.

# Libs

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.pipeline import Pipeline

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Embedding,  Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import RMSprop

from tensorflow.data import Dataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer, StandardScaler
from tensorflow import keras
from sklearn import metrics
from sklearn.impute import SimpleImputer

from sklearn.model_selection import GridSearchCV, cross_val_score

# Load Dataset

This step we simply get our data to our working environment. Because we are not dealing with live data, a simple pandas usage is enough.

In [2]:
%%time

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

Wall time: 14 ms


# Preprocessing

In [3]:
%%time

train['Survived'] = train['Survived'].astype(str)

train['n_missing'] = train.isna().sum(axis=1)
test['n_missing'] = test.isna().sum(axis=1)

test['Pclass']= test['Pclass'].astype(str)
test['Pclass']= test['Pclass'].astype(str)

features = [col for col in train.columns if col not in ['Survived', 'PassengerId']]

Wall time: 5.99 ms


### *Name* Column 

In [4]:
print(len(train['Name'].unique()))
print(train['Name'].unique()[0:5])

891
['Braund, Mr. Owen Harris'
 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)'
 'Heikkinen, Miss. Laina' 'Futrelle, Mrs. Jacques Heath (Lily May Peel)'
 'Allen, Mr. William Henry']


**With the *Name* column the way it is, we can't use it in our models.** The reason is because as every person has a unique name, then the name has no information about our variable of interest (*Survived*).

One thing we can see in this column is the presence of titles. **We can probably assume different survival rates when considering different titles.**

In [5]:
name_and_title = [name.split(", ")[1] for name in train['Name']]
title = [name.split(".")[0] for name in name_and_title]
print(len(title))

891


In [6]:
print(len(np.unique(title)))
np.unique(title)

17


array(['Capt', 'Col', 'Don', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Master',
       'Miss', 'Mlle', 'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir',
       'the Countess'], dtype='<U12')

In [7]:
train['Name'] = title
test['Name'] = [name.split(".")[0] for name in [name.split(", ")[1] for name in test['Name']]] 

In [8]:
train = pd.concat([train, pd.get_dummies(train['Name']).filter(['Miss', 'Mr', 'Mrs', 'Ms'])], axis = 1)
train.drop('Name', axis = 1, inplace = True)

test = pd.concat([test, pd.get_dummies(test['Name']).filter(['Miss', 'Mr', 'Mrs', 'Ms'])], axis = 1)
test.drop('Name', axis = 1, inplace = True)

### Dealing with the Ticket feature

In [9]:
train['Ticket'][0:5]

0           A/5 21171
1            PC 17599
2    STON/O2. 3101282
3              113803
4              373450
Name: Ticket, dtype: object

**One hypothesis we can make** is that the numbers don't contain any relevant information and the prefix may contain relevant information.

In [10]:
ticket_prefixes = [ticket.split()[0] for ticket in train['Ticket']]
ticket_prefixes[0:5]

['A/5', 'PC', 'STON/O2.', '113803', '373450']

In [11]:
for i in range(len(ticket_prefixes)):
    try: 
        int(ticket_prefixes[i])
        ticket_prefixes[i] = "number_only"
    
    except Exception:
        pass

In [12]:
ticket_prefixes[0:5]

['A/5', 'PC', 'STON/O2.', 'number_only', 'number_only']

In [13]:
print(len(np.unique(ticket_prefixes)))
np.unique(ticket_prefixes)

44


array(['A./5.', 'A.5.', 'A/4', 'A/4.', 'A/5', 'A/5.', 'A/S', 'A4.', 'C',
       'C.A.', 'C.A./SOTON', 'CA', 'CA.', 'F.C.', 'F.C.C.', 'Fa', 'LINE',
       'P/PP', 'PC', 'PP', 'S.C./A.4.', 'S.C./PARIS', 'S.O./P.P.',
       'S.O.C.', 'S.O.P.', 'S.P.', 'S.W./PP', 'SC', 'SC/AH', 'SC/PARIS',
       'SC/Paris', 'SCO/W', 'SO/C', 'SOTON/O.Q.', 'SOTON/O2', 'SOTON/OQ',
       'STON/O', 'STON/O2.', 'SW/PP', 'W./C.', 'W.E.P.', 'W/C', 'WE/P',
       'number_only'], dtype='<U11')

In [14]:
ticket_prefixes = [s.replace(".", "") for s in ticket_prefixes]
ticket_prefixes = [s.replace(",", "") for s in ticket_prefixes]
ticket_prefixes = [s.upper() for s in ticket_prefixes]

In [15]:
print(len(np.unique(ticket_prefixes)))
np.unique(ticket_prefixes)

34


array(['A/4', 'A/5', 'A/S', 'A4', 'A5', 'C', 'CA', 'CA/SOTON', 'FA', 'FC',
       'FCC', 'LINE', 'NUMBER_ONLY', 'P/PP', 'PC', 'PP', 'SC', 'SC/A4',
       'SC/AH', 'SC/PARIS', 'SCO/W', 'SO/C', 'SO/PP', 'SOC', 'SOP',
       'SOTON/O2', 'SOTON/OQ', 'SP', 'STON/O', 'STON/O2', 'SW/PP', 'W/C',
       'WE/P', 'WEP'], dtype='<U11')

In [16]:
test_ticket_prefixes = [ticket.split()[0] for ticket in test['Ticket']]
for i in range(len(test_ticket_prefixes)):
    try: 
        int(test_ticket_prefixes[i])
        test_ticket_prefixes[i] = "number_only"
    
    except Exception:
        pass

test_ticket_prefixes = [s.replace(".", "") for s in test_ticket_prefixes]
test_ticket_prefixes = [s.replace(",", "") for s in test_ticket_prefixes]
test_ticket_prefixes = [s.upper() for s in test_ticket_prefixes]

In [17]:
train['Ticket'] = ticket_prefixes
test['Ticket'] = test_ticket_prefixes

In [18]:
train = pd.concat([train, pd.get_dummies(train['Ticket']).filter(['PC', 'CA', 'NUMBER_ONLY'])], axis = 1)
train.drop('Ticket', axis = 1, inplace = True)

test = pd.concat([test, pd.get_dummies(test['Ticket']).filter(['PC', 'CA', 'NUMBER_ONLY'])], axis = 1)
test.drop('Ticket', axis = 1, inplace = True)

### Dealing with the Cabin feature

Same as the *Ticket* feature. I will assume that the number doesn't have relevant information.

In [19]:
train['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [20]:
cabin_prefix = []
for i in range(len(train['Cabin'])):
    try:
        cabin_prefix.append(train['Cabin'][i][0: 1: 1])
    
    except:
        cabin_prefix.append(train['Cabin'][i])        

In [21]:
np.unique(cabin_prefix)

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'nan'], dtype='<U32')

In [22]:
cabin_test_prefix = []
for i in range(len(test['Cabin'])):
    try:
        cabin_test_prefix.append(test['Cabin'][i][0: 1: 1])
    
    except:
        cabin_test_prefix.append(test['Cabin'][i])

In [23]:
train['Cabin'] = cabin_prefix
test['Cabin'] = cabin_test_prefix

In [24]:
train = pd.concat([train, pd.get_dummies(train['Cabin']).filter(['NaN', 'B', 'C'])], axis = 1)
train.drop('Cabin', axis = 1, inplace = True)

test = pd.concat([test, pd.get_dummies(test['Cabin']).filter(['NaN', 'B', 'C'])], axis = 1)
test.drop('Cabin', axis = 1, inplace = True)

## Pclass, Sex and Embarked variables

In [25]:
train = pd.get_dummies(train, columns = ['Pclass', 'Sex', 'Embarked'])
test = pd.get_dummies(test, columns = ['Pclass', 'Sex', 'Embarked'])

## Imputer and Scaler

In [26]:
%%time

features = [col for col in train.columns if col not in ['Survived', 'PassengerId']]
numerical_features = [col for col in features if col in ['Age', 'SibSp', 'Parch', 'Fare', 'n_missing']]

pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='mean',missing_values=np.nan)),
        ("scaler", StandardScaler())
        ])

train[numerical_features] = pipe.fit_transform(train[numerical_features])
test[numerical_features] = pipe.transform(test[numerical_features])

Wall time: 12 ms


# Base Models

## Light GBM

In [64]:
import lightgbm as lgb
import optuna

def objective(trial):
    
    train_x, valid_x, train_y, valid_y = train_test_split(train[features], train['Survived'], test_size=0.20)
    dtrain = lgb.Dataset(train_x, label=train_y)

    # 2. Suggest values of the hyperparameters using a trial object.
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = np.rint(gbm.predict(valid_x))
    
    accuracy = metrics.accuracy_score(valid_y, preds)
    return accuracy
    

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

[I 2021-12-20 20:01:47,356] A new study created in memory with name: no-name-ea31ee86-b719-48ba-8aa9-aaa3a685a0ce
[I 2021-12-20 20:01:47,432] Trial 0 finished with value: 0.8212290502793296 and parameters: {'lambda_l1': 4.014087766874642e-08, 'lambda_l2': 0.00015744713645363923, 'num_leaves': 27, 'feature_fraction': 0.9039739089558969, 'bagging_fraction': 0.5226995774038986, 'bagging_freq': 2, 'min_child_samples': 14}. Best is trial 0 with value: 0.8212290502793296.
[I 2021-12-20 20:01:47,483] Trial 1 finished with value: 0.8268156424581006 and parameters: {'lambda_l1': 0.1672344251180262, 'lambda_l2': 0.7843167242876422, 'num_leaves': 53, 'feature_fraction': 0.8166852384968912, 'bagging_fraction': 0.7811594127210835, 'bagging_freq': 3, 'min_child_samples': 41}. Best is trial 1 with value: 0.8268156424581006.
[I 2021-12-20 20:01:47,526] Trial 2 finished with value: 0.8156424581005587 and parameters: {'lambda_l1': 0.00017708841586685798, 'lambda_l2': 0.00031156625260156764, 'num_leaves'

[I 2021-12-20 20:01:49,070] Trial 22 finished with value: 0.8156424581005587 and parameters: {'lambda_l1': 4.456210459282396e-05, 'lambda_l2': 9.01589611112316e-06, 'num_leaves': 144, 'feature_fraction': 0.5900388157058891, 'bagging_fraction': 0.5909910965605213, 'bagging_freq': 7, 'min_child_samples': 19}. Best is trial 17 with value: 0.888268156424581.
[I 2021-12-20 20:01:49,154] Trial 23 finished with value: 0.8491620111731844 and parameters: {'lambda_l1': 1.3278365624841614e-06, 'lambda_l2': 9.66978833334972e-07, 'num_leaves': 93, 'feature_fraction': 0.65611927475749, 'bagging_fraction': 0.4859082547661398, 'bagging_freq': 6, 'min_child_samples': 46}. Best is trial 17 with value: 0.888268156424581.
[I 2021-12-20 20:01:49,235] Trial 24 finished with value: 0.7821229050279329 and parameters: {'lambda_l1': 1.5118636760227041e-06, 'lambda_l2': 4.741791220877294e-07, 'num_leaves': 91, 'feature_fraction': 0.6830975755832005, 'bagging_fraction': 0.4772723498874582, 'bagging_freq': 6, 'min

[I 2021-12-20 20:01:51,196] Trial 45 finished with value: 0.8268156424581006 and parameters: {'lambda_l1': 3.211467322693401e-08, 'lambda_l2': 3.2897077953322542e-06, 'num_leaves': 151, 'feature_fraction': 0.7430258322057448, 'bagging_fraction': 0.9087981081638211, 'bagging_freq': 3, 'min_child_samples': 41}. Best is trial 17 with value: 0.888268156424581.
[I 2021-12-20 20:01:51,286] Trial 46 finished with value: 0.8659217877094972 and parameters: {'lambda_l1': 1.3034820165067492e-07, 'lambda_l2': 0.006216751799674592, 'num_leaves': 221, 'feature_fraction': 0.887900669052977, 'bagging_fraction': 0.8822779254045133, 'bagging_freq': 4, 'min_child_samples': 52}. Best is trial 17 with value: 0.888268156424581.
[I 2021-12-20 20:01:51,370] Trial 47 finished with value: 0.8379888268156425 and parameters: {'lambda_l1': 2.635809084276726e-07, 'lambda_l2': 0.008879952194204354, 'num_leaves': 220, 'feature_fraction': 0.8865378310045782, 'bagging_fraction': 0.8847093382860047, 'bagging_freq': 4, '

[I 2021-12-20 20:01:53,408] Trial 68 finished with value: 0.8491620111731844 and parameters: {'lambda_l1': 1.8140344430762188e-06, 'lambda_l2': 0.0027438866043373407, 'num_leaves': 160, 'feature_fraction': 0.773979048213384, 'bagging_fraction': 0.6695430555998325, 'bagging_freq': 6, 'min_child_samples': 32}. Best is trial 17 with value: 0.888268156424581.
[I 2021-12-20 20:01:53,508] Trial 69 finished with value: 0.8491620111731844 and parameters: {'lambda_l1': 0.0021718118431612032, 'lambda_l2': 1.1529553248243765, 'num_leaves': 115, 'feature_fraction': 0.5501104762130927, 'bagging_fraction': 0.9697494729677064, 'bagging_freq': 2, 'min_child_samples': 38}. Best is trial 17 with value: 0.888268156424581.
[I 2021-12-20 20:01:53,614] Trial 70 finished with value: 0.8100558659217877 and parameters: {'lambda_l1': 4.471531116534123e-08, 'lambda_l2': 3.2801735253452864e-07, 'num_leaves': 97, 'feature_fraction': 0.5140462204030041, 'bagging_fraction': 0.5791843211083372, 'bagging_freq': 4, 'mi

[I 2021-12-20 20:01:55,555] Trial 91 finished with value: 0.8212290502793296 and parameters: {'lambda_l1': 1.6548445650932262e-05, 'lambda_l2': 0.000719858742598174, 'num_leaves': 179, 'feature_fraction': 0.883737826676938, 'bagging_fraction': 0.44651305979878936, 'bagging_freq': 1, 'min_child_samples': 29}. Best is trial 17 with value: 0.888268156424581.
[I 2021-12-20 20:01:55,649] Trial 92 finished with value: 0.8156424581005587 and parameters: {'lambda_l1': 4.20733353753976e-06, 'lambda_l2': 0.0012452764271294675, 'num_leaves': 189, 'feature_fraction': 0.8968371219295614, 'bagging_fraction': 0.4633428993270486, 'bagging_freq': 3, 'min_child_samples': 35}. Best is trial 17 with value: 0.888268156424581.
[I 2021-12-20 20:01:55,752] Trial 93 finished with value: 0.7821229050279329 and parameters: {'lambda_l1': 2.052634006531029e-06, 'lambda_l2': 0.0004672383009780988, 'num_leaves': 174, 'feature_fraction': 0.8536946487972983, 'bagging_fraction': 0.42411235525558, 'bagging_freq': 1, 'mi

...

[I 2021-12-20 20:03:43,127] Trial 891 finished with value: 0.8100558659217877 and parameters: {'lambda_l1': 3.5985684143891178e-06, 'lambda_l2': 6.515414082308973e-05, 'num_leaves': 200, 'feature_fraction': 0.8551400035141034, 'bagging_fraction': 0.4803739858936273, 'bagging_freq': 2, 'min_child_samples': 32}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:43,304] Trial 892 finished with value: 0.8212290502793296 and parameters: {'lambda_l1': 5.416305646999922e-07, 'lambda_l2': 4.8539932224633144e-05, 'num_leaves': 250, 'feature_fraction': 0.8733646167810598, 'bagging_fraction': 0.9380005477838985, 'bagging_freq': 4, 'min_child_samples': 30}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:43,450] Trial 893 finished with value: 0.8547486033519553 and parameters: {'lambda_l1': 4.1228632744642873e-07, 'lambda_l2': 0.00013071868291082424, 'num_leaves': 193, 'feature_fraction': 0.5583626992410219, 'bagging_fraction': 0.41020287500709673, 'baggin

[I 2021-12-20 20:03:46,604] Trial 913 finished with value: 0.8491620111731844 and parameters: {'lambda_l1': 8.684105478390072e-07, 'lambda_l2': 2.3412755242383778e-05, 'num_leaves': 220, 'feature_fraction': 0.911881412640447, 'bagging_fraction': 0.8904839525844483, 'bagging_freq': 6, 'min_child_samples': 34}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:46,742] Trial 914 finished with value: 0.8268156424581006 and parameters: {'lambda_l1': 1.7582327557269384e-06, 'lambda_l2': 9.10266497630376e-06, 'num_leaves': 167, 'feature_fraction': 0.6139083157592625, 'bagging_fraction': 0.42258275093971376, 'bagging_freq': 4, 'min_child_samples': 65}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:46,911] Trial 915 finished with value: 0.8435754189944135 and parameters: {'lambda_l1': 7.535574529659975e-07, 'lambda_l2': 0.00021633345455412815, 'num_leaves': 186, 'feature_fraction': 0.8219478436279973, 'bagging_fraction': 0.9474794301979691, 'bagging_f

[I 2021-12-20 20:03:50,005] Trial 935 finished with value: 0.8324022346368715 and parameters: {'lambda_l1': 2.564392732996956e-08, 'lambda_l2': 2.442753515993745e-05, 'num_leaves': 247, 'feature_fraction': 0.8404488939505308, 'bagging_fraction': 0.9126365961546246, 'bagging_freq': 4, 'min_child_samples': 21}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:50,171] Trial 936 finished with value: 0.8324022346368715 and parameters: {'lambda_l1': 9.918905796420264e-07, 'lambda_l2': 5.871022940038232e-07, 'num_leaves': 181, 'feature_fraction': 0.8032378475147829, 'bagging_fraction': 0.9050027621834802, 'bagging_freq': 6, 'min_child_samples': 32}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:50,317] Trial 937 finished with value: 0.8659217877094972 and parameters: {'lambda_l1': 2.171943608919896e-06, 'lambda_l2': 7.730804418663933e-05, 'num_leaves': 199, 'feature_fraction': 0.8464795292438796, 'bagging_fraction': 0.9296277264723557, 'bagging_fre

[I 2021-12-20 20:03:53,406] Trial 957 finished with value: 0.8435754189944135 and parameters: {'lambda_l1': 6.256497688051129e-08, 'lambda_l2': 3.75010784781064e-05, 'num_leaves': 229, 'feature_fraction': 0.8785852712800358, 'bagging_fraction': 0.8726518106407952, 'bagging_freq': 6, 'min_child_samples': 28}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:53,559] Trial 958 finished with value: 0.8659217877094972 and parameters: {'lambda_l1': 4.313059001082937e-06, 'lambda_l2': 1.9724625305171913e-05, 'num_leaves': 213, 'feature_fraction': 0.6377344275925888, 'bagging_fraction': 0.8308658283073886, 'bagging_freq': 6, 'min_child_samples': 39}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:53,699] Trial 959 finished with value: 0.8379888268156425 and parameters: {'lambda_l1': 1.332678661531028e-07, 'lambda_l2': 2.151144223231594e-05, 'num_leaves': 216, 'feature_fraction': 0.8753359660824006, 'bagging_fraction': 0.4960567724242642, 'bagging_fre

[I 2021-12-20 20:03:57,009] Trial 979 finished with value: 0.7821229050279329 and parameters: {'lambda_l1': 8.70302559694915e-07, 'lambda_l2': 0.0001924808872388547, 'num_leaves': 192, 'feature_fraction': 0.8342658179565876, 'bagging_fraction': 0.9339260566620124, 'bagging_freq': 6, 'min_child_samples': 29}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:57,155] Trial 980 finished with value: 0.8100558659217877 and parameters: {'lambda_l1': 0.002304336643965012, 'lambda_l2': 3.475482964081446e-05, 'num_leaves': 159, 'feature_fraction': 0.712315287834934, 'bagging_fraction': 0.8860390831427523, 'bagging_freq': 4, 'min_child_samples': 87}. Best is trial 883 with value: 0.8938547486033519.
[I 2021-12-20 20:03:57,320] Trial 981 finished with value: 0.8547486033519553 and parameters: {'lambda_l1': 1.9661305469648983e-08, 'lambda_l2': 1.0690472946949061e-05, 'num_leaves': 224, 'feature_fraction': 0.8028828859617084, 'bagging_fraction': 0.8955693095249072, 'bagging_freq

In [63]:
train['Survived'] = train['Survived'].astype(bool)

In [38]:
gbm = lgb.train(study.best_params, lgb.Dataset(train[features], label=train['Survived']))
preds = gbm.predict(test[features])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000302 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 251
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 21
[LightGBM] [Info] Start training from score 0.383838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [58]:
preds

array([ 2.21659333e-02,  3.82187672e-01,  6.42803378e-02,  5.07251034e-01,
        5.79591610e-01,  1.18345910e-01,  3.36342599e-01,  1.22575728e-01,
        9.89043988e-01,  2.43881707e-02,  2.35722943e-04,  3.07501256e-02,
        8.81203844e-01, -3.16674225e-02,  8.44469255e-01,  9.01964762e-01,
        1.40891336e-02,  3.96641495e-01,  5.91768275e-01,  6.55683457e-01,
        9.76641670e-02,  6.42143931e-01,  1.07855649e+00,  2.70761765e-01,
        1.11442121e+00, -1.07561684e-01,  1.13918835e+00,  4.33530760e-01,
        7.23131821e-01,  3.57533320e-01,  1.41931785e-02,  2.85584602e-01,
        5.04265215e-01,  2.43546911e-01,  5.24281226e-01,  2.35121588e-01,
        8.96427546e-02,  1.54611515e-01, -1.59076991e-02,  5.87113762e-01,
        5.61627558e-02,  6.64026419e-01,  2.68481435e-01,  1.14920994e+00,
        9.87404625e-01,  1.47149225e-01,  2.69159497e-01,  1.58082097e-01,
        1.03412977e+00,  7.42811739e-01,  4.11175118e-01,  7.65222233e-02,
        9.10101990e-01,  

In [46]:
submission = pd.read_csv('data/submission.csv')

In [48]:
submission['Survived'] = np.abs(np.rint(preds))

In [50]:
submission.to_csv("data/submission.csv", index = False)

In [ ]:
train.describe()

#### Here we already have some important information: 
- Low sample dataset: **891** observations (some models are not appropriate for low sample datasets)
- Low dimensionality: 12 columns (11 without the index)
  - Before even checking for redundant columns.
  - Not much information to work on.
- A lot of qualitative features.
- Need for scaling.

# Data Cleaning and Transformation
In this step we transform our current dataframe to a usable one. Here we are looking for *missings*, *corrupted*, or *imprecise* information. Later we try to create other variables to help our model performance.

## Transforming categorical data

**We scraped every title with those two lists comprehension.**

## Encoding

In [ ]:
train['Pclass'], test['Pclass'] = train['Pclass'].astype(str), test['Pclass'].astype(str)
features = [col for col in train.columns if col not in ['PassengerId', 'Survived']]
categorical_features = [col for col in train.columns if col in ['Pclass', 'Name','Sex','Ticket','Cabin','Embarked']]
numerical_features = [col for col in train.columns if col in ['Age','SibSp','Parch', 'Fare']]

In [ ]:
from category_encoders import TargetEncoder
encoder = TargetEncoder()

train_target_encoding = encoder.fit_transform(train[categorical_features], train['Survived'])
test_target_encoding = encoder.transform(test[categorical_features])

In [ ]:
new_train = pd.concat([train['Survived'], train[numerical_features], train_target_encoding], axis = 1)
new_test = pd.concat([test[numerical_features], test_target_encoding], axis = 1)

## Imputer

In [ ]:
from sklearn.impute import KNNImputer
knni = KNNImputer()
features = [col for col in new_train.columns if col not in ['PassengerId', 'Survived']]

new_train[features] = knni.fit_transform(new_train[features])
new_test[features] = knni.transform(new_test[features])

## Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

new_train[features] = std_scaler.fit_transform(new_train[features])
new_test[features] = std_scaler.transform(new_test[features])

# Model Building

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

gb_classif = GradientBoostingClassifier()

In [ ]:
gb_classif.fit(new_train[features], new_train['Survived'])
cv = cross_validate(gb_classif, new_train[features], new_train['Survived'], cv = 5)

In [ ]:
cv['test_score'].mean()

In [ ]:
pd.DataFrame(gb_classif.predict(new_test[features])).to_csv("submission.csv")

## Categorical features encoding, dealing with missing values and scaling variables
Most models don't work with categorical features. Here I will test two ways: **One-Hot Encoding** and **Target Encoding**. 

For replacing numerical missing values, I will test imputting **mean**, **median** values, as well a **linear regression** algorithm.

For replacing categorical missing values, I will test imputting **mode** values, as well a **kNN** algorithm.

For scaling the variable, I will use test both **MinMaxScaler** and **StandardScaler**. 

**A class will be used so I can easily try different dataprep tools.**

## Encoding

In [ ]:




train_name_dummies = pd.get_dummies(train['Name'], columns = categorical_features, dummy_na = True).filter(['Mr','Mrs','Miss','Master'])
test_name_dummies = pd.get_dummies(test['Name'], columns = categorical_features, dummy_na = True).filter(['Mr','Mrs','Miss','Master'])

train_pclass_dummies = pd.get_dummies(train['Pclass'], columns = categorical_features, dummy_na = True)
test_pclass_dummies = pd.get_dummies(test['Pclass'], columns = categorical_features, dummy_na = True)

train_sex_dummies = pd.get_dummies(train['Sex'], columns = categorical_features, dummy_na = True)
test_sex_dummies = pd.get_dummies(test['Sex'], columns = categorical_features, dummy_na = True)


train_ticket_dummies = pd.get_dummies(train['Ticket'], columns = categorical_features, dummy_na = True).filter(['NUMBER_ONLY'])
test_ticket_dummies = pd.get_dummies(test['Ticket'], columns = categorical_features, dummy_na = True).filter(['NUMBER_ONLY'])

train_cabin_dummies = pd.get_dummies(train['Cabin'], columns = categorical_features, dummy_na = True).filter(['NaN'])
test_cabin_dummies = pd.get_dummies(test['Cabin'], columns = categorical_features, dummy_na = True).filter(['NaN'])

train_embarked_dummies = pd.get_dummies(train['Embarked'], columns = categorical_features, dummy_na = True).filter(['Q','S','C'])
test_embarked_dummies = pd.get_dummies(test['Embarked'], columns = categorical_features, dummy_na = True).filter(['Q','S','C'])

## Imputer and Scaler

In [ ]:
OneHotEncoder() == OneHotEncoder

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from category_encoders import TargetEncoder


features = [col for col in train.columns if col not in ['PassengerId', 'Survived']]
categorical_features = [col for col in train.columns if col in ['Pclass', 'Name','Sex','Ticket','Cabin','Embarked']]
numerical_features = [col for col in train.columns if col in ['Age','SibSp','Parch', 'Fare']]

class DataPrep():
    
    def __init__(self, imputer, scaler, encoder):
        self.num_imputer = num_imputer
        self.scaler = scaler
        self.encoder = encoder
        
        self.pipe = Pipeline([
            ('numeric_imputer', self.imputer),
            ('scaler', self.scaler)            
        ])
        
        if type(encoder) == OneHotEncoder:
            self.new_features = 
            
    def fit(self, X):
        return(self.pipe.fit(X[numerical_features]))
        
    def transform(self, X):
        new_df = pd.concat(X[cat_features])
        return(self.pipe.transform(X[numerical_features]))
    
    
    def fit_transform(self, X):
        
        new_cat_X = pd.DataFrame(np.array(self.pipe_1.fit_transform(X[categorical_features])))
        new_num_X = pd.DataFrame(np.array(self.pipe_2.fit_transform(X[numerical_features])))
        
        return(pd.concat([new_cat_X.reset_index(drop=True), new_num_X.reset_index(drop=True)], axis=1).reset_index(drop=True))

In [ ]:
dataprep = DataPrep(
    imputer = SimpleImputer(strategy = 'mean'),
    scaler = StandardScaler()
)

In [ ]:
onehot.fit_transform(train[categorical_features]).shape
onehot.categories_

In [ ]:
new_train = dataprep.fit_transform(train)
new_test = dataprep.transform(test)

In [ ]:
new_train = pd.concat([train[numerical_features].reset_index(drop=True), new_train_cat_features], axis=1)
new_test = pd.concat([test[numerical_features].reset_index(drop=True), new_test_cat_features], axis=1)

In [ ]:
new_train.columns

In [ ]:
# Ignore observations of missing values if we are dealing with large data sets and less number of records has missing values
# Ignore variable, if it is not significant
# Develop model to predict missing values
# Treat missing data as just another category

## Dealing with *missings*

In [ ]:
train.isna().sum()
test.isna().sum()

In [ ]:
import missingno as msno
msno.matrix(new_train);

### Missings

In [ ]:
for col in train.columns:
    print(f"Feature: {col} {train[col].isna().sum()}")

We will have to deal with **2** main problems here: Age and Cabin. We will decide what to do with the *Embarked* missing values later.

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='mean', missing_values=np.nan)),
        ("scaler", QuantileTransformer(n_quantiles=64,output_distribution='uniform')),
        ('bin', KBinsDiscretizer(n_bins=64, encode='ordinal',strategy='uniform'))
        ])